In [1]:
# !pip install mxnet-mkl==1.6.0 numpy==1.23.1

In [1]:
# !pip install elephas
# 
# exit() # restart the runtime to refresh installed pkg

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.2/380.2 kB 1.6 MB/s eta 0:00:00m eta 0:00:010:01:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [854 lines of output]
           ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 5.2 MB/s eta 0:00:00
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached Cython-0.29.33-cp310-cp310-manylinux_2_17_x86_64.manyli

In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import *
from pyspark.ml.regression import *
from pyspark.ml.evaluation import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import seaborn as sns

: 

In [ ]:
# import zipfile
# with zipfile.ZipFile("data/ba-walmart.zip") as zf:
#     zf.extractall(
#         "data")

In [2]:
import seaborn as sns

In [3]:
spark = SparkSession.builder.appName("Vu dep trai").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/04 05:17:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc= spark.sparkContext

In [5]:
# test = pd.read_csv('data/ba-walmart/test.csv')
# train = pd.read_csv('data/ba-walmart/train.csv')
# features = pd.read_csv('data/ba-walmart/features.csv')
# stores = pd.read_csv('data/ba-walmart/stores.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/data/ba-walmart/test.csv'

In [8]:
# features.isna().sum()

In [5]:
df_stores_raw = spark.read.csv("data/ba-walmart/stores.csv", header=True, inferSchema=True)
df_stores_raw.show(5)

+-----+----+------+
|Store|Type|  Size|
+-----+----+------+
|    1|   A|151315|
|    2|   A|202307|
|    3|   B| 37392|
|    4|   A|205863|
|    5|   B| 34875|
+-----+----+------+
only showing top 5 rows



In [6]:
df_feature_raw = spark.read.csv("data/ba-walmart/features.csv", header=True, inferSchema=True)
df_feature_raw.show(5)

+-----+-------------------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
|Store|               Date|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|        CPI|Unemployment|IsHoliday|
+-----+-------------------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
|    1|2010-02-05 00:00:00|      42.31|     2.572|       NA|       NA|       NA|       NA|       NA|211.0963582|       8.106|    false|
|    1|2010-02-12 00:00:00|      38.51|     2.548|       NA|       NA|       NA|       NA|       NA|211.2421698|       8.106|     true|
|    1|2010-02-19 00:00:00|      39.93|     2.514|       NA|       NA|       NA|       NA|       NA|211.2891429|       8.106|    false|
|    1|2010-02-26 00:00:00|      46.63|     2.561|       NA|       NA|       NA|       NA|       NA|211.3196429|       8.106|    false|
|    1|2010-03-05 00:00:00|       46.5|     2.62

In [7]:
df_train_raw = spark.read.csv("data/ba-walmart/train.csv", header=True, inferSchema=True)
df_test_raw = spark.read.csv("data/ba-walmart/test.csv", header=True, inferSchema=True)
df_train_raw.show(5)

+-----+----+-------------------+------------+---------+
|Store|Dept|               Date|Weekly_Sales|IsHoliday|
+-----+----+-------------------+------------+---------+
|    1|   1|2010-02-05 00:00:00|     24924.5|    false|
|    1|   1|2010-02-12 00:00:00|    46039.49|     true|
|    1|   1|2010-02-19 00:00:00|    41595.55|    false|
|    1|   1|2010-02-26 00:00:00|    19403.54|    false|
|    1|   1|2010-03-05 00:00:00|     21827.9|    false|
+-----+----+-------------------+------------+---------+
only showing top 5 rows



In [8]:
#df_feature_raw drop Markdown
df_feature = df_feature_raw.drop("MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5")
df_feature.show(5)

+-----+-------------------+-----------+----------+-----------+------------+---------+
|Store|               Date|Temperature|Fuel_Price|        CPI|Unemployment|IsHoliday|
+-----+-------------------+-----------+----------+-----------+------------+---------+
|    1|2010-02-05 00:00:00|      42.31|     2.572|211.0963582|       8.106|    false|
|    1|2010-02-12 00:00:00|      38.51|     2.548|211.2421698|       8.106|     true|
|    1|2010-02-19 00:00:00|      39.93|     2.514|211.2891429|       8.106|    false|
|    1|2010-02-26 00:00:00|      46.63|     2.561|211.3196429|       8.106|    false|
|    1|2010-03-05 00:00:00|       46.5|     2.625|211.3501429|       8.106|    false|
+-----+-------------------+-----------+----------+-----------+------------+---------+
only showing top 5 rows



In [9]:
# merge df_train and df_feature, df_store, join left
df_train = df_train_raw.join(df_feature, how="left", on=["Store", "Date", "IsHoliday"], ).join(df_stores_raw, how="left", on=["Store"])
df_test = df_test_raw.join(df_feature, how="left", on=["Store", "Date", "IsHoliday"]).join(df_stores_raw, how="left", on=["Store"])
# convert CPI, Unemployment to float
df_train = df_train.withColumn("CPI", df_train["CPI"].cast(FloatType())).withColumn("Unemployment", df_train["Unemployment"].cast(DoubleType()))
df_test = df_test.withColumn("CPI", df_test["CPI"].cast(FloatType())).withColumn("Unemployment", df_test["Unemployment"].cast(DoubleType()))
# summary info df_train
df_train.printSchema()
df_train.show(5)


root
 |-- Store: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- IsHoliday: boolean (nullable = true)
 |-- Dept: integer (nullable = true)
 |-- Weekly_Sales: double (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Fuel_Price: double (nullable = true)
 |-- CPI: float (nullable = true)
 |-- Unemployment: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Size: integer (nullable = true)

+-----+-------------------+---------+----+------------+-----------+----------+---------+------------+----+------+
|Store|               Date|IsHoliday|Dept|Weekly_Sales|Temperature|Fuel_Price|      CPI|Unemployment|Type|  Size|
+-----+-------------------+---------+----+------------+-----------+----------+---------+------------+----+------+
|    1|2010-02-05 00:00:00|    false|   1|     24924.5|      42.31|     2.572|211.09636|       8.106|   A|151315|
|    1|2010-02-12 00:00:00|     true|   1|    46039.49|      38.51|     2.548|211.24217|       8.106| 

In [10]:
df_train = df_train.withColumn("Year", year("Date")).withColumn("Month", month("Date")).withColumn("Week", weekofyear("Date"))
df_test = df_test.withColumn("Year", year("Date")).withColumn("Month", month("Date")).withColumn("Week", weekofyear("Date"))
df_train = df_train.withColumn("IsHoliday", df_train["IsHoliday"].cast(IntegerType()))
df_test = df_test.withColumn("IsHoliday", df_test["IsHoliday"].cast(IntegerType()))
df_train.createOrReplaceTempView("df_train")
df_test.createOrReplaceTempView("df_test")
df_train.show(5)

+-----+-------------------+---------+----+------------+-----------+----------+---------+------------+----+------+----+-----+----+
|Store|               Date|IsHoliday|Dept|Weekly_Sales|Temperature|Fuel_Price|      CPI|Unemployment|Type|  Size|Year|Month|Week|
+-----+-------------------+---------+----+------------+-----------+----------+---------+------------+----+------+----+-----+----+
|    1|2010-02-05 00:00:00|        0|   1|     24924.5|      42.31|     2.572|211.09636|       8.106|   A|151315|2010|    2|   5|
|    1|2010-02-12 00:00:00|        1|   1|    46039.49|      38.51|     2.548|211.24217|       8.106|   A|151315|2010|    2|   6|
|    1|2010-02-19 00:00:00|        0|   1|    41595.55|      39.93|     2.514|211.28914|       8.106|   A|151315|2010|    2|   7|
|    1|2010-02-26 00:00:00|        0|   1|    19403.54|      46.63|     2.561|211.31964|       8.106|   A|151315|2010|    2|   8|
|    1|2010-03-05 00:00:00|        0|   1|     21827.9|       46.5|     2.625|211.35014|  

In [11]:
# # draw pie chart of store type given by df_stores
# df_stores = df_stores_raw.toPandas()
# df_stores["Type"].value_counts().plot.pie(autopct="%1.1f%%", figsize=(6, 6), labels=None).set_title("Store Type")
# plt.axis("off")
# plt.legend(labels=df_stores["Type"].value_counts().index, loc="upper right")

In [12]:
df_train_pd = df_train.toPandas()

/opt/bitnami/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [13]:
df_train_pd.isna().sum()

Store           0
Date            0
IsHoliday       0
Dept            0
Weekly_Sales    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
Type            0
Size            0
Year            0
Month           0
Week            0
dtype: int64

In [14]:
#sns.scatterplot(x="Size", y="Weekly_Sales", data=df_train_pd, hue="Type", alpha=0.2).set_title("Size, Type vs Weekly_Sales")

In [15]:
# df_train_pd.groupby("IsHoliday")["Weekly_Sales"].mean().plot.bar().set_title("IsHoliday vs Weekly_Sales")

In [16]:
# sns.scatterplot(x="Fuel_Price", y="Weekly_Sales", hue="Type", data=df_train_pd, alpha=0.2).set_title("Fuel_Price vs Weekly_Sales")

In [17]:
# find relationship between Unemployment and Weekly_Sales
# sns.scatterplot(x="Unemployment", y="Weekly_Sales", hue="Type", data=df_train_pd, alpha=0.2).set_title("Unemployment vs Weekly_Sales")


In [18]:
# find relationship between CPI and Weekly_Sales
# sns.scatterplot(x="CPI", y="Weekly_Sales", hue="Type", data=df_train_pd, alpha=0.2).set_title("CPI vs Weekly_Sales")

In [19]:
# find relationship between Temperature and Weekly_Sales
# sns.scatterplot(x="Temperature", y="Weekly_Sales", hue="Type", data=df_train_pd, alpha=0.2).set_title("Temperature vs Weekly_Sales")

In [20]:
# average_sales_by_time = spark.sql("SELECT Year, Month, AVG(Weekly_Sales) AS Average_Sales FROM df_train GROUP BY Year, Month ORDER BY Year, Month")
# average_sales_by_time_pd = average_sales_by_time.toPandas()
# sns.barplot(x="Month", y="Average_Sales", hue='Year', data=average_sales_by_time_pd).set_title("Average Sales by Month and Year")
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [21]:
types = df_train.select("Type").distinct().collect()
types.sort()
mapping = {t.Type: str(i) for i, t in enumerate(types)}
df_train = df_train.replace(mapping, subset=["Type"])
df_test = df_test.replace(mapping, subset=["Type"])
df_train = df_train.withColumn("Type", df_train["Type"].cast(IntegerType()))
df_test = df_test.withColumn("Type", df_test["Type"].cast(IntegerType()))
df_train_pd = df_train.toPandas()
df_train.show(5)

/opt/bitnami/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


+-----+-------------------+---------+----+------------+-----------+----------+---------+------------+----+------+----+-----+----+
|Store|               Date|IsHoliday|Dept|Weekly_Sales|Temperature|Fuel_Price|      CPI|Unemployment|Type|  Size|Year|Month|Week|
+-----+-------------------+---------+----+------------+-----------+----------+---------+------------+----+------+----+-----+----+
|    1|2010-02-05 00:00:00|        0|   1|     24924.5|      42.31|     2.572|211.09636|       8.106|   0|151315|2010|    2|   5|
|    1|2010-02-12 00:00:00|        1|   1|    46039.49|      38.51|     2.548|211.24217|       8.106|   0|151315|2010|    2|   6|
|    1|2010-02-19 00:00:00|        0|   1|    41595.55|      39.93|     2.514|211.28914|       8.106|   0|151315|2010|    2|   7|
|    1|2010-02-26 00:00:00|        0|   1|    19403.54|      46.63|     2.561|211.31964|       8.106|   0|151315|2010|    2|   8|
|    1|2010-03-05 00:00:00|        0|   1|     21827.9|       46.5|     2.625|211.35014|  

In [22]:
# find correlation between attributes and Weekly_Sales
results = df_train_pd.corr()["Weekly_Sales"].sort_values(ascending=False)
print(results)


Weekly_Sales    1.000000
Size            0.243828
Dept            0.148032
Month           0.028409
Week            0.027673
IsHoliday       0.012774
Fuel_Price     -0.000120
Temperature    -0.002312
Year           -0.010111
CPI            -0.020921
Unemployment   -0.025864
Store          -0.085195
Type           -0.182242
Name: Weekly_Sales, dtype: float64


/tmp/ipykernel_6000/3010913235.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  results = df_train_pd.corr()["Weekly_Sales"].sort_values(ascending=False)


In [23]:
keep_cols = results[results.abs() > 0.01].index.values
print("Training attribute: ", keep_cols)
ignore_attributes = set(df_train_pd.columns) - set(keep_cols)
print("Ignore attribute: ", ignore_attributes)


Training attribute:  ['Weekly_Sales' 'Size' 'Dept' 'Month' 'Week' 'IsHoliday' 'Year' 'CPI'
 'Unemployment' 'Store' 'Type']
Ignore attribute:  {'Temperature', 'Fuel_Price', 'Date'}


In [24]:
df_train_clean = df_train.drop(*ignore_attributes)
df_test_clean = df_test.drop(*ignore_attributes)


In [25]:
df_train_clean.show(50, truncate= False)

+-----+---------+----+------------+---------+------------+----+------+----+-----+----+
|Store|IsHoliday|Dept|Weekly_Sales|CPI      |Unemployment|Type|Size  |Year|Month|Week|
+-----+---------+----+------------+---------+------------+----+------+----+-----+----+
|1    |0        |1   |24924.5     |211.09636|8.106       |0   |151315|2010|2    |5   |
|1    |1        |1   |46039.49    |211.24217|8.106       |0   |151315|2010|2    |6   |
|1    |0        |1   |41595.55    |211.28914|8.106       |0   |151315|2010|2    |7   |
|1    |0        |1   |19403.54    |211.31964|8.106       |0   |151315|2010|2    |8   |
|1    |0        |1   |21827.9     |211.35014|8.106       |0   |151315|2010|3    |9   |
|1    |0        |1   |21043.39    |211.38065|8.106       |0   |151315|2010|3    |10  |
|1    |0        |1   |22136.64    |211.21564|8.106       |0   |151315|2010|3    |11  |
|1    |0        |1   |26229.21    |211.01804|8.106       |0   |151315|2010|3    |12  |
|1    |0        |1   |57258.43    |210.8204

In [26]:
# (trainingDF, validationDF) = train_df.withColumn("label", train_df["class"].cast(DoubleType())+lit(1)).select("features", "label").randomSplit([0.9, 0.1])

In [27]:
# train_df.withColumn("label", train_df["class"].cast(DoubleType())+lit(1)).select("features", "label").show(truncate=False)

In [28]:
# trainingDF.show(50,truncate=False)

In [29]:
df_train =df_train.drop("Date")
df_train =df_train.drop("Unemployment")
df_train =df_train.drop("CPI")


In [30]:
df_train.show()

+-----+---------+----+------------+-----------+----------+----+------+----+-----+----+
|Store|IsHoliday|Dept|Weekly_Sales|Temperature|Fuel_Price|Type|  Size|Year|Month|Week|
+-----+---------+----+------------+-----------+----------+----+------+----+-----+----+
|    1|        0|   1|     24924.5|      42.31|     2.572|   0|151315|2010|    2|   5|
|    1|        1|   1|    46039.49|      38.51|     2.548|   0|151315|2010|    2|   6|
|    1|        0|   1|    41595.55|      39.93|     2.514|   0|151315|2010|    2|   7|
|    1|        0|   1|    19403.54|      46.63|     2.561|   0|151315|2010|    2|   8|
|    1|        0|   1|     21827.9|       46.5|     2.625|   0|151315|2010|    3|   9|
|    1|        0|   1|    21043.39|      57.79|     2.667|   0|151315|2010|    3|  10|
|    1|        0|   1|    22136.64|      54.58|      2.72|   0|151315|2010|    3|  11|
|    1|        0|   1|    26229.21|      51.45|     2.732|   0|151315|2010|    3|  12|
|    1|        0|   1|    57258.43|      62

In [31]:
# create regression model
input_cols = df_train.columns
# input_cols = ['CPI', 'Unemployment', 'IsHoliday', 'Week','Weekly_Sales']
input_cols.remove("Weekly_Sales")
output_col = "Weekly_Sales"
assembler = VectorAssembler( outputCol="preFeatures")
assembler.setInputCols(input_cols)
assembler.setHandleInvalid("skip")
#df_train-_clean_ = df_train_clean_.withColumn("weight", when(df_train_clean_["IsHoliday"] == 1, 5).otherwise(1))
assemble_train_df = assembler.transform(df_train)


In [32]:
# check df_val has nan
df_train_pd = df_train.toPandas()
df_train_pd.isna().sum()

Store           0
IsHoliday       0
Dept            0
Weekly_Sales    0
Temperature     0
Fuel_Price      0
Type            0
Size            0
Year            0
Month           0
Week            0
dtype: int64

In [33]:
(df_train_clean_, df_val) = assemble_train_df.select("preFeatures", "Weekly_Sales").randomSplit([0.8, 0.2], seed=42)

saler = MinMaxScaler(inputCol="preFeatures", outputCol="features")
saler = saler.fit(df_train_clean_)
scaledData = saler.transform(df_train_clean_)
scaledData_valid = saler.transform(df_val)
# scaler = MinMaxScaler(inputCol="preFeatures", outputCol="features")
# # rescale each feature to range [min, max].
# scaledData = scaler.fit(df_train_clean_).transform(df_train_clean_).select("features", "Weekly_Sales")
# scaledData_valid = scaler.fit(df_train_clean_).transform(df_val).select("features", "Weekly_Sales")

In [34]:
scaledData.show()

+--------------------+------------+--------------------+
|         preFeatures|Weekly_Sales|            features|
+--------------------+------------+--------------------+
|[1.0,0.0,1.0,35.4...|     17359.7|(10,[3,4,6,7,9],[...|
|[1.0,0.0,1.0,39.9...|    41595.55|(10,[3,4,6,8,9],[...|
|[1.0,0.0,1.0,42.3...|     24924.5|(10,[3,4,6,8,9],[...|
|[1.0,0.0,1.0,43.8...|    18461.18|(10,[3,4,6,7,9],[...|
|[1.0,0.0,1.0,43.9...|    33305.92|[0.0,0.0,0.0,0.45...|
|[1.0,0.0,1.0,45.3...|     54060.1|[0.0,0.0,0.0,0.46...|
|[1.0,0.0,1.0,46.5...|     21827.9|(10,[3,4,6,8,9],[...|
|[1.0,0.0,1.0,46.6...|    19403.54|(10,[3,4,6,8,9],[...|
|[1.0,0.0,1.0,47.9...|    46788.75|[0.0,0.0,0.0,0.48...|
|[1.0,0.0,1.0,48.2...|    15984.24|(10,[3,4,6,7],[0....|
|[1.0,0.0,1.0,48.9...|    25293.49|[0.0,0.0,0.0,0.49...|
|[1.0,0.0,1.0,49.0...|    16567.69|(10,[3,4,6,7],[0....|
|[1.0,0.0,1.0,49.2...|    22517.56|(10,[3,4,6,8,9],[...|
|[1.0,0.0,1.0,49.8...|    44912.86|(10,[3,4,6,8,9],[...|
|[1.0,0.0,1.0,51.4...|    19552

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
# from tensorflow.keras.utils import to_categorical, generic_utils

# nb_classes = train_df.select("category").distinct().count()
input_dim = len(scaledData.select("features").first()[0])

model = Sequential()
model.add(Dense(8, input_shape=(input_dim,)))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('linear'))

model.compile(loss='mean_absolute_error', optimizer='adam')

2023-02-04 05:18:01.313139: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-04 05:18:01.456124: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bitnami/python/lib:/opt/bitnami/spark/venv/lib/python3.8/site-packages/numpy.libs:
2023-02-04 05:18:01.456155: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-04 05:18:01.475625: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS

In [36]:


# # Initialize SparkML Estimator and set all relevant properties
# estimator = ElephasEstimator()
# estimator.setFeaturesCol("scaled_features")             # These two come directly from pyspark,
# estimator.setLabelCol("index_category")                 # hence the camel case. Sorry :)
# estimator.set_keras_model_config(model.to_yaml())       # Provide serialized Keras model
# estimator.set_categorical_labels(True)
# estimator.set_nb_classes(nb_classes)
# estimator.set_num_workers(1)  # We just use one worker here. Feel free to adapt it.
# estimator.set_epochs(20) 
# estimator.set_batch_size(128)
# estimator.set_verbosity(1)
# estimator.set_validation_split(0.15)
# estimator.set_optimizer_config(opt_conf)
# estimator.set_mode("synchronous")
# estimator.set_loss("categorical_crossentropy")
# estimator.set_metrics(['acc'])

In [37]:
from elephas.ml_model import ElephasEstimator
from tensorflow.keras import optimizers
# from tensorflow.keras import optimizers


adam = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(adam)
# Initialize Spark ML Estimator
estimator = ElephasEstimator()
estimator.set_keras_model_config(model.to_json())
estimator.set_optimizer_config(opt_conf)
estimator.setLabelCol("Weekly_Sales") 
estimator.set_mode("synchronous")
estimator.set_loss("mae")
estimator.set_metrics(['mse'])
estimator.set_epochs(5)
estimator.set_batch_size(32)
estimator.set_validation_split(0.1)
estimator.set_categorical_labels(False)



/opt/bitnami/python/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ElephasEstimator_72bdf11503e8

In [38]:
fitted_pipeline = estimator.fit(scaledData)

>>> Fit model


2023-02-04 05:18:11.053089: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-04 05:18:11.063117: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-04 05:18:11.064439: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

>>> Synchronous training complete.


In [39]:
# check scaledData_valid has nan
scaledData_valid.show()

+--------------------+------------+--------------------+
|         preFeatures|Weekly_Sales|            features|
+--------------------+------------+--------------------+
|[1.0,0.0,1.0,42.2...|    21665.76|[0.0,0.0,0.0,0.43...|
|[1.0,0.0,1.0,44.0...|    17341.47|(10,[3,4,6,7,9],[...|
|[1.0,0.0,1.0,46.3...|    31497.65|(10,[3,4,6,8,9],[...|
|[1.0,0.0,1.0,48.5...|     16894.4|(10,[3,4,6,7,9],[...|
|[1.0,0.0,1.0,51.4...|    26229.21|(10,[3,4,6,8,9],[...|
|[1.0,0.0,1.0,54.1...|     18365.1|(10,[3,4,6,7,9],[...|
|[1.0,0.0,1.0,57.3...|    46845.87|[0.0,0.0,0.0,0.58...|
|[1.0,0.0,1.0,59.5...|    20327.61|[0.0,0.0,0.0,0.60...|
|[1.0,0.0,1.0,64.6...|    20138.19|[0.0,0.0,0.0,0.65...|
|[1.0,0.0,1.0,64.7...|    22366.88|[0.0,0.0,0.0,0.65...|
|[1.0,0.0,1.0,64.8...|    16145.35|(10,[3,4,6,8,9],[...|
|[1.0,0.0,1.0,65.9...|     22107.7|[0.0,0.0,0.0,0.66...|
|[1.0,0.0,1.0,66.5...|     31579.9|[0.0,0.0,0.0,0.67...|
|[1.0,0.0,1.0,67.4...|    16555.11|(10,[3,4,6,8,9],[...|
|[1.0,0.0,1.0,69.1...|    27390

In [40]:
# Evaluate Spark model by evaluating the underlying model
score = fitted_pipeline.transform(scaledData_valid).select("prediction", "Weekly_Sales").rdd.map(lambda x: (x[0], x[1])).collect()
# prediction = fitted_pipeline.transform(scaledData)


#
# A fatal error has been detected by the Java Runtime Environment:
#
#  SIGSEGV (0xb) at pc=0x00007f1348e5a2f8, pid=6030, tid=7515
#
# JRE version: OpenJDK Runtime Environment (11.0.16+8) (build 11.0.16+8-post-Debian-1deb11u1)
# Java VM: OpenJDK 64-Bit Server VM (11.0.16+8-post-Debian-1deb11u1, mixed mode, sharing, tiered, compressed oops, g1 gc, linux-amd64)
# Problematic frame:
# J 17431 c2 org.apache.spark.sql.catalyst.expressions.UnsafeRow.getArray(I)Lorg/apache/spark/sql/catalyst/util/ArrayData; (6 bytes) @ 0x00007f1348e5a2f8 [0x00007f1348e5a1c0+0x0000000000000138]
#
# Core dump will be written. Default location: Core dumps may be processed with "/usr/share/apport/apport -p%p -s%s -c%c -d%d -P%P -u%u -g%g -- %E" (or dumping to /home/gianglt2/project/BA/spark/core.6030)
#
# An error report file with more information is saved as:
# /home/gianglt2/project/BA/spark/hs_err_pid6030.log
[thread 7527 also had an error]
Compiled method (c1)   76713 20596       3       org.apache.spark.sq

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 40422)
Traceback (most recent call last):
  File "/opt/bitnami/python/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/bitnami/python/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/bitnami/python/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/bitnami/python/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/opt/bitnami/spark/python/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/opt/bitnami/spark/python/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/opt/bitnami/spark/python/pyspark/accumulators.py", line 257, in accum_updates
    num_updates = read_int(self.r

ConnectionRefusedError: [Errno 111] Connection refused

In [27]:
prediction.show()

23/02/04 04:55:51 ERROR PythonRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 666, in main
    eval_type = read_int(infile)
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:765)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:747)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at org.apache.spark.rdd.RDD$$anon$3.hasNext(RDD.scala:943)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection

Py4JJavaError: An error occurred while calling o328.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 53.0 failed 1 times, most recent failure: Lost task 0.0 in stage 53.0 (TID 83) (9878222aad48 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3838/0x000000084149f840: (struct<Store_double_VectorAssembler_ee238b6831b1:double,IsHoliday_double_VectorAssembler_ee238b6831b1:double,Dept_double_VectorAssembler_ee238b6831b1:double,Temperature:double,Fuel_Price:double,CPI_double_VectorAssembler_ee238b6831b1:double,Unemployment:double,Type_double_VectorAssembler_ee238b6831b1:double,Size_double_VectorAssembler_ee238b6831b1:double,Year_double_VectorAssembler_ee238b6831b1:double,Month_double_VectorAssembler_ee238b6831b1:double,Week_double_VectorAssembler_ee238b6831b1:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryE